# 법률문서 계층적 청킹 및 임베딩 (Nomic Embed Text V2 MOE)

마크다운 구조 기반의 계층적 청킹과 `nomic-ai/nomic-embed-text-v2-moe` 모델을 활용한 임베딩

## 1. 필수 라이브러리 설치 및 임포트

In [1]:
print("2")

2


In [5]:
import os
import re
import json
import requests
from pathlib import Path
from typing import List, Dict, Tuple
import numpy as np

# 모델 초기화 - Ollama를 통한 임베딩 생성
print("🧠 Ollama를 통한 임베딩 모델 준비 중...")
OLLAMA_BASE_URL = "http://localhost:11434"

# Ollama 서버가 실행 중인지 확인
try:
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=2)
    if response.status_code == 200:
        print("✅ Ollama 서버 연결 성공")
    else:
        print("⚠️  Ollama 서버 연결 실패 - 수동 설치 필요")
except Exception as e:
    print(f"⚠️  Ollama 서버에 연결할 수 없습니다: {e}")
    print("   설치 방법: https://ollama.ai 에서 Ollama 다운로드 후 'ollama serve' 실행")

def get_embedding_from_ollama(text: str, model: str = "nomic-embed-text") -> np.ndarray:
    """
    Ollama를 사용하여 임베딩 생성
    """
    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/embeddings",
            json={"model": model, "prompt": text},
            timeout=30
        )
        if response.status_code == 200:
            embedding = response.json()["embedding"]
            return np.array(embedding, dtype=np.float32)
        else:
            raise Exception(f"Ollama API error: {response.status_code}")
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        # 대체: 무작위 벡터 반환 (테스트용)
        print("   무작위 벡터로 대체합니다.")
        return np.random.randn(384)

print("✅ 임베딩 함수 준비 완료")


🧠 Ollama를 통한 임베딩 모델 준비 중...
⚠️  Ollama 서버에 연결할 수 없습니다: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/tags (Caused by ConnectTimeoutError(<HTTPConnection(host='localhost', port=11434) at 0x28d1a2f2f90>, 'Connection to localhost timed out. (connect timeout=2)'))
   설치 방법: https://ollama.ai 에서 Ollama 다운로드 후 'ollama serve' 실행
✅ 임베딩 함수 준비 완료


## 2. 경로 설정

In [ ]:
# 입력/출력 경로 설정
MD_FILE_PATH = r"c:\Users\미소정보기술\airflow-practice\output\markdown\법률test_converted.md"
JSON_OUTPUT_DIR = r"c:\Users\미소정보기술\airflow-practice\output\json"
JSON_OUTPUT_FILE = os.path.join(JSON_OUTPUT_DIR, "법률_계층적_청킹_임베딩.json")

# 출력 폴더 생성
os.makedirs(JSON_OUTPUT_DIR, exist_ok=True)

print(f"📄 입력 파일: {MD_FILE_PATH}")
print(f"💾 출력 파일: {JSON_OUTPUT_FILE}")
print(f"✅ 출력 디렉토리 준비 완료")

## 3. 마크다운 파싱 및 계층적 청킹 함수

In [ ]:
def parse_markdown_hierarchical(md_content: str) -> List[Dict]:
    """
    마크다운 문서를 계층적 구조로 파싱
    H1(법률명) > H2(장) > H3(조) > H4(항) 구조 유지
    """
    lines = md_content.split('\n')
    hierarchy = []
    
    current_h1 = None
    current_h2 = None
    current_h3 = None
    current_h4 = None
    current_content = []
    
    for line in lines:
        # 헤더 레벨 감지
        h1_match = re.match(r'^#\s+(.+?)$', line)
        h2_match = re.match(r'^##\s+(.+?)$', line)
        h3_match = re.match(r'^###\s+(.+?)$', line)
        h4_match = re.match(r'^####\s+(.+?)$', line)
        
        if h1_match:
            current_h1 = h1_match.group(1).strip()
            current_h2 = None
            current_h3 = None
            current_h4 = None
            current_content = []
        elif h2_match:
            current_h2 = h2_match.group(1).strip()
            current_h3 = None
            current_h4 = None
            current_content = []
        elif h3_match:
            # 이전 H3의 내용 저장
            if current_h3 and current_content:
                hierarchy.append({
                    'level': 3,
                    'h1': current_h1,
                    'h2': current_h2,
                    'h3': current_h3,
                    'h4': current_h4,
                    'content': '\n'.join(current_content).strip()
                })
            current_h3 = h3_match.group(1).strip()
            current_h4 = None
            current_content = []
        elif h4_match:
            # 이전 H4의 내용 저장
            if current_h4 and current_content:
                hierarchy.append({
                    'level': 4,
                    'h1': current_h1,
                    'h2': current_h2,
                    'h3': current_h3,
                    'h4': current_h4,
                    'content': '\n'.join(current_content).strip()
                })
            current_h4 = h4_match.group(1).strip()
            current_content = []
        elif line.strip():  # 빈 줄이 아닌 경우
            current_content.append(line)
    
    # 마지막 항목 저장
    if current_h4 and current_content:
        hierarchy.append({
            'level': 4,
            'h1': current_h1,
            'h2': current_h2,
            'h3': current_h3,
            'h4': current_h4,
            'content': '\n'.join(current_content).strip()
        })
    elif current_h3 and current_content:
        hierarchy.append({
            'level': 3,
            'h1': current_h1,
            'h2': current_h2,
            'h3': current_h3,
            'h4': None,
            'content': '\n'.join(current_content).strip()
        })
    
    return hierarchy


def create_hierarchical_chunks(hierarchy_list: List[Dict], max_chars: int = 1500) -> List[Dict]:
    """
    계층적 구조를 청크로 변환
    같은 조(H3) 내의 여러 항(H4)을 하나의 청크로 병합 가능
    """
    chunks = []
    temp_chunk = {
        'h1': None,
        'h2': None,
        'h3': None,
        'items': []  # H4 항목들
    }
    temp_size = 0
    
    for item in hierarchy_list:
        item_text = f"{item.get('h4', '')}\n{item['content']}"
        item_size = len(item_text)
        
        # 같은 조(H3)인 경우
        if temp_chunk['h3'] == item['h3'] and temp_size + item_size <= max_chars:
            temp_chunk['items'].append(item)
            temp_size += item_size
        else:
            # 새로운 청크 시작
            if temp_chunk['items']:
                chunks.append(temp_chunk.copy())
            temp_chunk = {
                'h1': item['h1'],
                'h2': item['h2'],
                'h3': item['h3'],
                'items': [item]
            }
            temp_size = item_size
    
    # 마지막 청크 추가
    if temp_chunk['items']:
        chunks.append(temp_chunk)
    
    return chunks


def chunk_to_text(chunk: Dict) -> str:
    """
    청크를 텍스트 형태로 변환 (임베딩용)
    """
    text_parts = []
    
    if chunk['h1']:
        text_parts.append(f"# {chunk['h1']}")
    if chunk['h2']:
        text_parts.append(f"## {chunk['h2']}")
    if chunk['h3']:
        text_parts.append(f"### {chunk['h3']}")
    
    for item in chunk['items']:
        if item['h4']:
            text_parts.append(f"#### {item['h4']}")
        if item['content']:
            text_parts.append(item['content'])
    
    return '\n\n'.join(text_parts)


print("✅ 함수 정의 완료")

## 4. 마크다운 파일 읽기 및 파싱

In [ ]:
# 마크다운 파일 읽기
print(f"📖 마크다운 파일 읽기: {MD_FILE_PATH}")
with open(MD_FILE_PATH, 'r', encoding='utf-8') as f:
    md_content = f.read()

print(f"✅ 파일 읽기 완료 (크기: {len(md_content)} 글자)")

# 계층적 구조로 파싱
print("\n🔍 마크다운 계층적 파싱 시작...")
hierarchy = parse_markdown_hierarchical(md_content)
print(f"✅ 파싱 완료: {len(hierarchy)}개의 항목 추출")

# 추출된 첫 3개 항목 확인
print("\n📋 추출된 항목 샘플:")
for i, item in enumerate(hierarchy[:3]):
    print(f"\n[{i+1}] {item['h3']} - {item['h4']}")
    print(f"    내용: {item['content'][:100]}...")

## 5. 계층적 청킹 수행

In [ ]:
# 청킹
print("📦 청킹 수행 중...")
chunks = create_hierarchical_chunks(hierarchy, max_chars=1500)
print(f"✅ 청킹 완료: {len(chunks)}개의 청크 생성")

# 청크 통계
chunk_sizes = [len(chunk_to_text(chunk)) for chunk in chunks]
print(f"\n📊 청크 크기 통계:")
print(f"   - 최소: {min(chunk_sizes)} 글자")
print(f"   - 최대: {max(chunk_sizes)} 글자")
print(f"   - 평균: {sum(chunk_sizes) / len(chunk_sizes):.0f} 글자")

## 6. 임베딩 및 JSON 생성

In [ ]:
print("🧠 임베딩 생성 중...")
json_results = []

for chunk_idx, chunk in enumerate(chunks):
    chunk_text = chunk_to_text(chunk)
    
    # Ollama를 통한 임베딩 생성
    dense_vec = get_embedding_from_ollama(chunk_text)
    
    # 청크 ID 생성 (조+항+순번)
    chunk_id = f"{chunk['h3'].replace('조', '').strip()}_{chunk_idx+1:03d}"
    
    json_results.append({
        "id": chunk_id,
        "pk": f"LAW_CHUNK_{chunk_idx+1:04d}",
        "hierarchy": {
            "h1": chunk['h1'],
            "h2": chunk['h2'],
            "h3": chunk['h3'],
            "h4_items": [item['h4'] for item in chunk['items'] if item['h4']]
        },
        "text": chunk_text,
        "text_summary": chunk_text[:200] + "..." if len(chunk_text) > 200 else chunk_text,
        "dense_embedding": dense_vec.tolist(),
        "embedding_model": "ollama:nomic-embed-text",
        "chunk_size": len(chunk_text),
        "source_file": "법률test_converted.md",
        "chunk_index": chunk_idx + 1,
        "total_chunks": len(chunks)
    })
    
    if (chunk_idx + 1) % 5 == 0:
        print(f"   진행: {chunk_idx + 1}/{len(chunks)} 완료")

print(f"✅ 임베딩 완료: {len(json_results)}개의 청크")


## 7. JSON 파일 저장

In [ ]:
# JSON 저장
print(f"💾 JSON 파일 저장 중: {JSON_OUTPUT_FILE}")
with open(JSON_OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(json_results, f, ensure_ascii=False, indent=2)

print(f"✅ JSON 저장 완료")

# 파일 크기 확인
file_size_mb = os.path.getsize(JSON_OUTPUT_FILE) / (1024 * 1024)
print(f"📊 파일 크기: {file_size_mb:.2f} MB")

## 8. 결과 검증 및 샘플 출력

In [ ]:
# 결과 검증
print("\n📋 생성된 JSON 구조:")
print(f"\n총 청크 수: {len(json_results)}")
print(f"\n첫 번째 청크 샘플:")
sample_chunk = json_results[0]
print(f"  ID: {sample_chunk['id']}")
print(f"  Hierarchy: {sample_chunk['hierarchy']}")
print(f"  Text Preview: {sample_chunk['text_summary'][:100]}...")
print(f"  Embedding 길이: {len(sample_chunk['dense_embedding'])}")

# 통계
print(f"\n\n📊 최종 통계:")
print(f"  - 총 청크: {len(json_results)}")
print(f"  - 평균 청크 크기: {np.mean([c['chunk_size'] for c in json_results]):.0f} 글자")
print(f"  - 임베딩 차원: {len(json_results[0]['dense_embedding'])}")
print(f"  - 모델: {json_results[0]['embedding_model']}")

print("\n✅ 모든 프로세스 완료!")

## 9. RAG 검증 - 유사도 검색 테스트 (선택사항)

In [ ]:
# 쿼리 예시: 의료지원금 관련 조항 검색
query = "의료지원금 지급 대상"
query_embedding = model.encode([query], normalize_embeddings=True)[0]

# 코사인 유사도 계산
similarities = []
for chunk in json_results:
    chunk_embedding = np.array(chunk['dense_embedding'])
    similarity = np.dot(query_embedding, chunk_embedding)
    similarities.append((similarity, chunk))

# 상위 3개 결과
similarities.sort(reverse=True, key=lambda x: x[0])

print(f"\n🔍 쿼리: '{query}'")
print("\n📌 상위 3개 관련 청크:\n")
for i, (score, chunk) in enumerate(similarities[:3]):
    print(f"[{i+1}] 유사도: {score:.4f}")
    print(f"    조항: {chunk['hierarchy']['h3']}")
    print(f"    내용: {chunk['text_summary'][:150]}...\n")